In [1]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from typing import List, Tuple
from preprocessing import (
    clsf_boz_cd, clsf_rrnr_dvcd, clsf_dmfr_dvcd, clsf_crd_grd, clsf_pdgr_cd,
    clsf_cntr_catg_cd, get_disc_factor_all, get_loss_adj
)

In [2]:
# 환경설정
pd.options.display.float_format = '{:,.0f}'.format
os.makedirs('result', exist_ok=True)

# 전역변수
FILE_PATH = Path('./data/현행추정부채_일반')

In [3]:
# 데이터 불러오기
## 익스포져
일반_원수_미경과보험료 = pd.read_excel(FILE_PATH / '일반_원수_미경과보험료.xlsx', dtype={'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'ARC_INPL_CD': str})
일반_출재_미경과보험료 = pd.read_excel(FILE_PATH / '일반_출재_미경과보험료.xlsx', dtype={'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'ARC_INPL_CD': str, 'T02_RN_RINSC_CD': str})
일반_원수_예정보험료 = pd.read_excel(FILE_PATH / '일반_원수_예정보험료.xlsx', dtype={'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'ARC_INPL_CD': str})
일반_출재_예정보험료 = pd.read_excel(FILE_PATH / '일반_출재_예정보험료.xlsx', dtype={'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'ARC_INPL_CD': str, 'T02_RN_RINSC_CD': str})
일반_원수_개별추산액 = pd.read_excel(FILE_PATH / '일반_원수_개별추산액.xlsx', dtype={'RRNR_DAT_DVCD': str, 'RRNR_DMFR_DVCD': str, 'ARC_INPL_CD': str})
일반_출재_개별추산액 = pd.read_excel(FILE_PATH / '일반_출재_개별추산액.xlsx', dtype={'RRNR_DAT_DVCD': str, 'RRNR_DMFR_DVCD': str, 'ARC_INPL_CD': str, 'T02_RN_RINSC_CD': str})
일반_원수_직전3년보험금손조비 = pd.read_excel(FILE_PATH / '일반_원수_직전3년보험금손조비.xlsx', dtype={'RRNR_DAT_DVCD': str, 'RRNR_DMFR_DVCD': str, 'ARC_INPL_CD': str})
일반_출재_직전3년보험금손조비 = pd.read_excel(FILE_PATH / '일반_출재_직전3년보험금손조비.xlsx', dtype={'RRNR_DAT_DVCD': str, 'RRNR_DMFR_DVCD': str, 'ARC_INPL_CD': str})


## 기초정보, 가정 등
일반_상품정보 = pd.read_excel(FILE_PATH / '일반_상품정보.xlsx', dtype={'PDC_CD': str, 'PDGR_CD': str})
일반_보험금진전추이 = pd.read_excel(FILE_PATH / '일반_보험금진전추이.xlsx', dtype={'PDGR_CD': str, 'AY': str})
일반_최종손해율 = pd.read_excel(FILE_PATH / '일반_최종손해율.xlsx', dtype={'RRNR_DAT_DVCD': str, 'PDGR_CD': str}) \
        .rename(columns= {'RRNR_DAT_DVCD': 'RRNR_DVCD'})

## 공통
국가그룹 = pd.read_excel(FILE_PATH / '국가그룹.xlsx', dtype={'CNTR_CATG_CD': str})
재보험자_국내신용등급 = pd.read_excel(FILE_PATH / '재보험자_국내신용등급.xlsx', dtype={'재보험사코드': str}) \
        .rename(columns = {'재보험사코드': 'T02_RN_RINSC_CD', '국내신용등급': 'CRD_GRD'})
선도부도율 = pd.read_excel(FILE_PATH / '선도부도율.xlsx')
할인율 = pd.read_excel(FILE_PATH / '할인율.xlsx')

In [4]:
# 데이터 전처리
## 미경과보험료
일반_원수_미경과보험료_가공 = 일반_원수_미경과보험료.copy()
일반_원수_미경과보험료_가공['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_원수_미경과보험료_가공, '원수')
일반_원수_미경과보험료_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_원수_미경과보험료_가공)
일반_원수_미경과보험료_가공['CNTR_CATG_CD'] = clsf_cntr_catg_cd(일반_원수_미경과보험료_가공, 국가그룹)
일반_원수_미경과보험료_가공['BOZ_CD'] = clsf_boz_cd(일반_원수_미경과보험료_가공, 일반_상품정보)
일반_원수_미경과보험료_가공['PDGR_CD'] = clsf_pdgr_cd(일반_원수_미경과보험료_가공, 일반_상품정보)
일반_원수_미경과보험료_가공['CRD_GRD'] = '#'
일반_원수_미경과보험료_가공.rename(columns = {'LTPD_URND_PRM': 'URND_PRM'}, inplace=True)
일반_원수_미경과보험료_가공.drop(['RRNR_DAT_DVCD', 'RRNR_CTC_BZ_DVCD', 'NTNL_CTRY_CD', 'ARC_INPL_CD'], axis=1, inplace=True)

일반_출재_미경과보험료_가공 = 일반_출재_미경과보험료.copy()
일반_출재_미경과보험료_가공['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_출재_미경과보험료_가공, '출재')
일반_출재_미경과보험료_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_출재_미경과보험료_가공)
일반_출재_미경과보험료_가공['CNTR_CATG_CD'] = clsf_cntr_catg_cd(일반_출재_미경과보험료_가공, 국가그룹)
일반_출재_미경과보험료_가공['BOZ_CD'] = clsf_boz_cd(일반_출재_미경과보험료_가공, 일반_상품정보)
일반_출재_미경과보험료_가공['PDGR_CD'] = clsf_pdgr_cd(일반_출재_미경과보험료_가공, 일반_상품정보)
일반_출재_미경과보험료_가공['CRD_GRD'] = clsf_crd_grd(일반_출재_미경과보험료_가공, 재보험자_국내신용등급)
일반_출재_미경과보험료_가공.rename(columns = {'T02_LTPD_RN_URND_PRM': 'URND_PRM'}, inplace=True)
일반_출재_미경과보험료_가공.drop(['RRNR_DAT_DVCD', 'RRNR_CTC_BZ_DVCD', 'ARC_INPL_CD', 'NTNL_CTRY_CD', 'T02_RN_RINSC_CD'], axis=1, inplace=True)

일반_미경과보험료_가공 = pd.concat([일반_원수_미경과보험료_가공, 일반_출재_미경과보험료_가공], axis=0).reset_index(drop=True)

## 예정보험료
일반_원수_예정보험료_가공 = 일반_원수_예정보험료.copy()
일반_원수_예정보험료_가공['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_원수_예정보험료_가공, '원수')
일반_원수_예정보험료_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_원수_예정보험료_가공)
일반_원수_예정보험료_가공['CNTR_CATG_CD'] = clsf_cntr_catg_cd(일반_원수_예정보험료_가공, 국가그룹)
일반_원수_예정보험료_가공['BOZ_CD'] = clsf_boz_cd(일반_원수_예정보험료_가공, 일반_상품정보)
일반_원수_예정보험료_가공['PDGR_CD'] = clsf_pdgr_cd(일반_원수_예정보험료_가공, 일반_상품정보)
일반_원수_예정보험료_가공['CRD_GRD'] = '#'
일반_원수_예정보험료_가공.rename(columns = {'WNCR_CNV_EPCT_PRM': 'EPCT_PRM'}, inplace=True)
일반_원수_예정보험료_가공.drop(['RRNR_DAT_DVCD', 'RRNR_CTC_BZ_DVCD', 'ARC_INPL_CD', 'NTNL_CTRY_CD'], axis=1, inplace=True)

일반_출재_예정보험료_가공 = 일반_출재_예정보험료.copy()
일반_출재_예정보험료_가공['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_출재_예정보험료_가공, '출재')
일반_출재_예정보험료_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_출재_예정보험료_가공)
일반_출재_예정보험료_가공['CNTR_CATG_CD'] = clsf_cntr_catg_cd(일반_출재_예정보험료_가공, 국가그룹)
일반_출재_예정보험료_가공['BOZ_CD'] = clsf_boz_cd(일반_출재_예정보험료_가공, 일반_상품정보)
일반_출재_예정보험료_가공['PDGR_CD'] = clsf_pdgr_cd(일반_출재_예정보험료_가공, 일반_상품정보)
일반_출재_예정보험료_가공['CRD_GRD'] = clsf_crd_grd(일반_출재_예정보험료_가공, 재보험자_국내신용등급)
일반_출재_예정보험료_가공.rename(columns = {'WNCR_CNV_RN_PRM': 'EPCT_PRM'}, inplace=True)
일반_출재_예정보험료_가공.drop(['ARC_INPL_CD', 'RRNR_CTC_BZ_DVCD', 'RRNR_DAT_DVCD', 'NTNL_CTRY_CD', 'T02_RN_RINSC_CD'], axis=1, inplace=True)

일반_예정보험료_가공 = pd.concat([일반_원수_예정보험료_가공, 일반_출재_예정보험료_가공], axis=0).reset_index(drop=True)

## 개별추산액
일반_원수_개별추산액_가공 = 일반_원수_개별추산액.copy()
일반_원수_개별추산액_가공['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_원수_개별추산액_가공, '원수')
일반_원수_개별추산액_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_원수_개별추산액_가공)
일반_원수_개별추산액_가공['CNTR_CATG_CD'] = clsf_cntr_catg_cd(일반_원수_개별추산액_가공, 국가그룹)
일반_원수_개별추산액_가공['BOZ_CD'] = clsf_boz_cd(일반_원수_개별추산액_가공, 일반_상품정보)
일반_원수_개별추산액_가공['PDGR_CD'] = clsf_pdgr_cd(일반_원수_개별추산액_가공, 일반_상품정보)
일반_원수_개별추산액_가공['CRD_GRD'] = '#'
일반_원수_개별추산액_가공.rename(columns = {'LTPD_OST_AMT': 'OST_AMT'}, inplace=True)
일반_원수_개별추산액_가공.drop(['RRNR_DAT_DVCD', 'RRNR_DMFR_DVCD', 'NTNL_CTRY_CD', 'ARC_INPL_CD'], axis=1, inplace=True)

일반_출재_개별추산액_가공 = 일반_출재_개별추산액.copy()
일반_출재_개별추산액_가공['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_출재_개별추산액_가공, '출재')
일반_출재_개별추산액_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_출재_개별추산액_가공)
일반_출재_개별추산액_가공['BOZ_CD'] = clsf_boz_cd(일반_출재_개별추산액_가공, 일반_상품정보)
일반_출재_개별추산액_가공['PDGR_CD'] = clsf_pdgr_cd(일반_출재_개별추산액_가공, 일반_상품정보)
일반_출재_개별추산액_가공['CRD_GRD'] = clsf_crd_grd(일반_출재_개별추산액_가공, 재보험자_국내신용등급)
일반_출재_개별추산액_가공.rename(columns = {'RN_LTPD_OST_AMT': 'OST_AMT'}, inplace=True)
일반_출재_개별추산액_가공.drop(['RRNR_DAT_DVCD', 'RRNR_DMFR_DVCD', 'NTNL_CTRY_CD', 'ARC_INPL_CD', 'T02_RN_RINSC_CD'], axis=1, inplace=True)

일반_개별추산액_가공 = pd.concat([일반_원수_개별추산액_가공, 일반_출재_개별추산액_가공], axis=0).reset_index(drop=True)

## 직전3년보험금손조비
일반_원수_직전3년보험금손조비_가공 = 일반_원수_직전3년보험금손조비.copy()
일반_원수_직전3년보험금손조비_가공['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_원수_직전3년보험금손조비_가공, '원수')
일반_원수_직전3년보험금손조비_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_원수_직전3년보험금손조비_가공)
일반_원수_직전3년보험금손조비_가공['BOZ_CD'] = clsf_boz_cd(일반_원수_직전3년보험금손조비_가공, 일반_상품정보)
일반_원수_직전3년보험금손조비_가공['PDGR_CD'] = clsf_pdgr_cd(일반_원수_직전3년보험금손조비_가공, 일반_상품정보)
일반_원수_직전3년보험금손조비_가공['CRD_GRD'] = '#'
일반_원수_직전3년보험금손조비_가공.rename(columns = {'ACCD_IVMT': 'DAG_IVMT', 'OGL_PYN_BNF': 'PYN_BNF'}, inplace=True)
일반_원수_직전3년보험금손조비_가공.drop(['RRNR_DAT_DVCD', 'RRNR_DMFR_DVCD', 'ARC_INPL_CD'], axis=1, inplace=True)

일반_출재_직전3년보험금손조비_가공 = 일반_출재_직전3년보험금손조비.copy()
일반_출재_직전3년보험금손조비_가공['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_출재_직전3년보험금손조비_가공, '출재')
일반_출재_직전3년보험금손조비_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_출재_직전3년보험금손조비_가공)
일반_출재_직전3년보험금손조비_가공['BOZ_CD'] = clsf_boz_cd(일반_출재_직전3년보험금손조비_가공, 일반_상품정보)
일반_출재_직전3년보험금손조비_가공['PDGR_CD'] = clsf_pdgr_cd(일반_출재_직전3년보험금손조비_가공, 일반_상품정보)
일반_출재_직전3년보험금손조비_가공['CRD_GRD'] = '#'
일반_출재_직전3년보험금손조비_가공.rename(columns = {'WNCR_CNV_RN_DAG_IVMT': 'DAG_IVMT', 'WNCR_CNV_PYN_BNF': 'PYN_BNF'}, inplace=True)
일반_출재_직전3년보험금손조비_가공.drop(['RRNR_DAT_DVCD', 'RRNR_DMFR_DVCD', 'ARC_INPL_CD'], axis=1, inplace=True)

일반_직전3년보험금손조비_가공 = pd.concat([일반_원수_직전3년보험금손조비_가공, 일반_출재_직전3년보험금손조비_가공], axis=0).reset_index(drop=True)

In [56]:
# 가정산출
## 최종손해율
### 8개 상품군(화재, 기술, 근재, 책임, 종합, 상해, 기타, 해상) 가정 사용
일반_최종손해율 = 일반_최종손해율.loc[lambda x: x['PDGR_CD'].isin(['23', '24', '25', '26', '27', '28', '29', '31'])]

## 손해조사비율
### 1. 원수+수재, 출재로 그룹핑
### 2. PDGR_CD 단위로 그룹핑
### 3. 해외의 경우 PDGR_CD 구분없이 산출
일반_손해조사비율 = 일반_직전3년보험금손조비_가공.copy()
일반_손해조사비율.loc[lambda x: x['RRNR_DVCD'].isin(['01', '02']), 'RRNR_DVCD'] = '04'
일반_손해조사비율.loc[lambda x: x['DMFR_DVCD']=='02', 'PDGR_CD'] = '#'
일반_손해조사비율 = 일반_손해조사비율.groupby(['RRNR_DVCD', 'DMFR_DVCD', 'PDGR_CD'])[['DAG_IVMT', 'PYN_BNF']].sum() \
        .eval('LAE_RATIO = DAG_IVMT/PYN_BNF') \
        .reset_index() \
        .drop(['DAG_IVMT', 'PYN_BNF'], axis=1)
일반_손해조사비율_국내 = 일반_손해조사비율.loc[lambda x: x['DMFR_DVCD']=='01']
일반_손해조사비율_해외 = pd.concat([일반_손해조사비율.loc[lambda x: x['DMFR_DVCD']=='02'].copy().assign(PDGR_CD = pdgr_cd) for pdgr_cd in ['23', '24', '25', '26', '27', '28', '29', '31']])
일반_손해조사비율 = pd.concat([일반_손해조사비율_국내, 일반_손해조사비율_해외], axis=0)
일반_원수_손해조사비율 = 일반_손해조사비율.loc[lambda x: x['RRNR_DVCD'] == '04'].copy().assign(RRNR_DVCD = '01')
일반_수재_손해조사비율 = 일반_손해조사비율.loc[lambda x: x['RRNR_DVCD'] == '04'].copy().assign(RRNR_DVCD = '02')
일반_출재_손해조사비율 = 일반_손해조사비율.query('RRNR_DVCD == "03"').copy()
일반_손해조사비율 = pd.concat([일반_원수_손해조사비율, 일반_수재_손해조사비율, 일반_출재_손해조사비율]).reset_index(drop=True)

In [93]:
# 데이터 집계
## 미경과보험료
일반_미경과보험료_집계 = 일반_미경과보험료_가공.copy()
# 일반_미경과보험료_집계.loc[lambda x: x['RRNR_DVCD'].isin(['01', '02']), 'RRNR_DVCD'] = '04'
# 일반_미경과보험료_집계.loc[lambda x: x['DMFR_DVCD'] == '02', 'BOZ_CD'] = '#'
일반_미경과보험료_집계 = 일반_미경과보험료_집계.groupby(['RRNR_DVCD', 'DMFR_DVCD', 'PDGR_CD', 'BOZ_CD'])['URND_PRM'].sum().reset_index()

## 예정보험료
일반_예정보험료_집계 = 일반_예정보험료_가공.copy()
# 일반_예정보험료_집계.loc[lambda x: x['RRNR_DVCD'].isin(['01', '02']), 'RRNR_DVCD'] = '04'
# 일반_예정보험료_집계.loc[lambda x: x['DMFR_DVCD'] == '02', 'BOZ_CD'] = '#'
일반_예정보험료_집계 = 일반_예정보험료_집계.groupby(['RRNR_DVCD', 'DMFR_DVCD', 'PDGR_CD', 'BOZ_CD'])['EPCT_PRM'].sum().reset_index()

## 보험료익스포져
일반_보험료익스포져 = 일반_미경과보험료_집계.merge(일반_예정보험료_집계, on=['RRNR_DVCD', 'DMFR_DVCD', 'PDGR_CD', 'BOZ_CD'], how='outer') \
        .assign(URND_PRM = lambda x: x['URND_PRM'].fillna(0).astype(float)) \
        .assign(EPCT_PRM = lambda x: x['EPCT_PRM'].fillna(0).astype(float))

## 할인요소
일반_할인요소 = get_disc_factor_all(일반_보험금진전추이, 할인율.query('KICS_SCEN_NO == 1'))

## 보험료부채
일반_보험료부채 = 일반_보험료익스포져 \
        .merge(일반_최종손해율, on=['RRNR_DVCD', 'PDGR_CD'], how='left') \
        .merge(일반_손해조사비율, on=['RRNR_DVCD', 'DMFR_DVCD', 'PDGR_CD'], how='left') \
        .merge(일반_할인요소[['PDGR_CD', 'DISC_FAC_PRM']], on=['PDGR_CD'], how='left') \
        .eval('FNAL_LOSS = (URND_PRM+EPCT_PRM)*FNAL_LSRT') \
        .eval('LAE = FNAL_LOSS*LAE_RATIO')

일반_보험료부채 = 일반_보험료부채.eval('MAINT_EXP_RATIO = 0.03') # 삭제예정
일반_보험료부채 = 일반_보험료부채.eval('MAINT_EXP = (URND_PRM+EPCT_PRM)*MAINT_EXP_RATIO') # 삭제예정

일반_보험료부채 = 일반_보험료부채 \
        .eval('LIAB_PRM = -EPCT_PRM+FNAL_LOSS+LAE+MAINT_EXP') \
        .eval('PV_LIAB_PRM = LIAB_PRM*DISC_FAC_PRM')
일반_보험료부채.loc[lambda x: x['RRNR_DVCD'].isin(['01', '02']), 'RRNR_DVCD'] = '04'
일반_보험료부채.loc[lambda x: x['DMFR_DVCD'] == '02', 'BOZ_CD'] = '#'
일반_보험료부채 = 일반_보험료부채 \
        .groupby(['RRNR_DVCD', 'DMFR_DVCD', 'BOZ_CD']) \
        [['LIAB_PRM', 'PV_LIAB_PRM']].sum() \
        .reset_index()
일반_보험료부채

,RRNR_DVCD,DMFR_DVCD,BOZ_CD,LIAB_PRM,PV_LIAB_PRM
0,03,01,A001,"1,681,429,490","1,675,059,797"
1,03,01,A002,"5,499,615,028","5,441,385,058"
2,03,01,A003,"23,630,978,379","23,496,717,004"
3,03,01,A004,"9,523,385,961","9,442,207,708"
4,03,01,A005,"744,080,127","733,112,900"
5,03,01,A006,"19,408,398,609","19,151,590,048"
6,03,01,A007,"23,811,871,935","23,683,563,151"
7,03,01,A009,"4,234,244,059","4,209,244,440"
8,03,01,A010,"28,871,612,801","28,701,150,421"
9,03,01,A011,"-4,695,897","-4,668,171"


In [54]:
# 데이터 내보내기